# Imports

In [ ]:
!pip install -q wandb
!pip install git+https://github.com/huggingface/transformers.git@main
!pip install -q datasets

  Cloning https://github.com/huggingface/transformers.git (to revision main) to /tmp/pip-req-build-13oph6z9
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-13oph6z9
  Resolved https://github.com/huggingface/transformers.git to commit 43bfd093e1817c0333a1e10fcbdd54f1032baad0
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
!pip list

Package                          Version
-------------------------------- ---------------------
absl-py                          1.4.0
aiohttp                          3.8.5
aiosignal                        1.3.1
alabaster                        0.7.13
albumentations                   1.3.1
altair                           4.2.2
anyio                            3.7.1
appdirs                          1.4.4
argon2-cffi                      23.1.0
argon2-cffi-bindings             21.2.0
array-record                     0.4.1
arviz                            0.15.1
astropy                          5.3.3
astunparse                       1.6.3
async-timeout                    4.0.3
attrs                            23.1.0
audioread                        3.0.1
autograd                         1.6.2
Babel                            2.12.1
backcall                         0.2.0
beautifulsoup4                   4.11.2
bleach                           6.0.0
blinker                          1.4
bl

In [ ]:
from google.colab import auth
from google.cloud import storage
import io
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, Add, Dense, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input as preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.python.keras import backend as K
from transformers import BertTokenizer, BertModel
from transformers import AutoTokenizer
from transformers import BlipProcessor, BlipForConditionalGeneration
import torch
import requests
from PIL import Image
from datasets import Dataset, load_dataset
import time

# W&B
import wandb
from wandb.keras import WandbCallback, WandbMetricsLogger

In [ ]:
wandb.login()

wandb: Currently logged in as: danhee (instacap). Use `wandb login --relogin` to force relogin


True

# CODE


In [ ]:
def get_dataset():
  def pull_data_from_gcs(project_id, bucket_name, credentials_file_path, data_file_path):
    %env GOOGLE_APPLICATION_CREDENTIALS $credentials_file_path

    client = storage.Client()
    bucket = client.get_bucket(bucket_name)
    blob = bucket.blob(data_file_path)

    csv_data = blob.download_as_text()

    df = pd.read_csv(io.StringIO(csv_data))

    return df

  def make_data_list(df, folder_name):
    data_list = []

    for index, row in df.iterrows():
      image_filename = row['image_file_name']  # Assuming this is the column name for image file names
      caption = row['caption']  # Assuming this is the column name for captions

      # Get image blob and download image data
      image_blob = bucket.blob(folder_name + image_filename)
      image_data = image_blob.download_as_bytes()

      # Open image using PIL
      image = Image.open(io.BytesIO(image_data))

      # Create dictionary and append to the list
      data_dict = {
          'image': image,
          'caption': caption
      }
      data_list.append(data_dict)

    return data_list

  def get_blip_transcription(data_list, blip_model_name='Salesforce/blip-image-captioning-base'):
    blip_processor = BlipProcessor.from_pretrained(blip_model_name)
    blip_model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")
    blip_preds = []
    for i, example in enumerate(data_list):
        image = example["image"]
        truth = example["caption"]
        inputs = blip_processor(images=image, return_tensors="pt")
        pixel_values = inputs.pixel_values
        generated_ids = blip_model.generate(
            pixel_values=pixel_values, max_length=50)
        generated_caption = blip_processor.batch_decode(
            generated_ids, skip_special_tokens=True)[0]
        blip_preds.append([image, truth, generated_caption])
    return blip_preds

  def get_raw_data(blip_preds):
    # seperate images, captions, blip transcriptions
    image_arrs = []  # image arrays (not PIL images)
    captions = []  # true captions
    transcriptions = []  # blip transcriptions
    for entry in blip_preds:
      # convert pil image to numpy array
      image_arrs.append(np.array(entry[0].resize((224, 224))))
      captions.append(entry[1])
      transcriptions.append(entry[2])

    return image_arrs, captions, transcriptions


  def extract_image_features(img_array, weights_name='imagenet'):
    # RESNET
    resnet_model = ResNet50(weights=weights_name, include_top=False, pooling='avg')
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    features = resnet_model.predict(img_array)
    return features

  # DON'T FORGET TO DO THIS:
  # image_features = [extract_image_features(img) for img in image_arrs]


  def get_bert_embeddings(blip_transcriptions, bert_model_name='bert-base-uncased'):
    bert_model = BertModel.from_pretrained(bert_model_name)
    tokenizer = AutoTokenizer.from_pretrained(bert_model_name)

    inputs = tokenizer(blip_transcriptions, return_tensors='pt',padding=True, truncation=True, max_length=128)
    outputs = bert_model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1)

    return embeddings

  # get blip embeddings DO ALTERELATLL ALTET LATER
  # transcription_embeddings = get_bert_embeddings(transcriptions)

  # DO THIS TOO:
  # transcription_embeddings = np.expand_dims(transcription_embeddings.detach().numpy(), axis = 1)

  def create_input_output_pairs(captions, image_features, transcription_embeddings):
    # Create a tokenizer and fit on the captions
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(captions)

    # Vocabulary size is the total number of unique words in your captions
    start = len(tokenizer.word_index) + 1
    stop = start + 1
    vocab_size = stop + 1

    # Convert captions to sequences of integer indices
    captions_sequences = tokenizer.texts_to_sequences(captions)

    # Insert start and stop sequences to the encoded caption size
    captions_sequences = [([start] + c) for c in captions_sequences]
    captions_sequences = [(c + [stop]) for c in captions_sequences]

    max_caption_length = max(len(c) for c in captions_sequences)

    # create input output pairs
    input_sequences = []
    output_sequences = []
    image_sequences = []
    transcription_sequences = []

    for j in range(len(captions_sequences)):
        caption_sequence = captions_sequences[j]
        for i in range(1, len(caption_sequence)):
            # Input sequence (words before the current word)
            input_seq = caption_sequence[:i]
            input_seq = pad_sequences([input_seq], max_caption_length)[0]
            # Output word (current word)
            output_word = caption_sequence[i]
            output_word = to_categorical(output_word, num_classes = vocab_size)
            input_sequences.append(input_seq)
            output_sequences.append(output_word)
            image_sequences.append(image_features[j])
            transcription_sequences.append(transcription_embeddings[j])

    input_sequences = np.array(input_sequences)
    input_sequences = np.expand_dims(input_sequences, axis = 1)
    output_sequences = np.array(output_sequences)
    image_sequences = np.reshape(image_sequences,(np.shape(image_sequences)[0], np.shape(image_sequences)[2]))
    transcription_sequences = np.reshape(transcription_sequences,(np.shape(transcription_sequences)[0], np.shape(transcription_sequences)[2]))

    return input_sequences, output_sequences, image_sequences, transcription_sequences, vocab_size, max_caption_length

  def get_train_test_split_datasets(n_posts, input_sequences, output_sequences, image_sequences, transcription_sequences, train_ratio=0.8):
    # train test split
    train_size = int(train_ratio * n_posts)
    test_size = n_posts - train_size

    train_dataset = tf.data.Dataset.from_tensor_slices(((image_sequences[:train_size], transcription_sequences[:train_size],input_sequences[:train_size]), output_sequences[:train_size]))
    test_dataset = tf.data.Dataset.from_tensor_slices(((image_sequences[train_size:], transcription_sequences[train_size:], input_sequences[train_size:]), output_sequences[train_size:]))

    train_dataset = train_dataset.shuffle(buffer_size=train_size).batch(batch_size)
    test_dataset = test_dataset.batch(batch_size)

    return train_dataset, test_dataset


  df = pull_data_from_gcs(project_id = 'ac215-project-398320', bucket_name='instacap-data', credentials_file_path = 'data_service_account.json', data_file_path='sample_dataset.csv')
  data_list = make_data_list(df, folder_name='posts/')
  blip_preds = get_blip_transcription(data_list)
  image_arrs, captions, transcriptions = get_raw_data(blip_preds)
  image_features = [extract_image_features(img) for img in image_arrs]
  transcription_embeddings = get_bert_embeddings(transcriptions)
  transcription_embeddings = np.expand_dims(transcription_embeddings.detach().numpy(), axis=1)
  input_sequences, output_sequences, image_sequences, transcription_sequences, vocab_size, max_caption_length = create_input_output_pairs(captions, image_features, transcription_embeddings)

  n_posts = len(data_list)

  train_dataset, test_dataset = get_train_test_split_datasets(n_posts, input_sequences, output_sequences, image_sequences, transcription_sequences)
  input_image_dim = image_features[0].shape[1]
  input_embedding_dim = transcription_embeddings.shape[-1]
  return train_dataset, test_dataset, max_caption_length, vocab_size, input_image_dim, input_embedding_dim


In [ ]:
def build_model(model_name, input_image_dim, input_embedding_dim, max_caption_length, vocab_size, n_units=256):
  # defining input layers:
  image_input = Input(shape=(input_image_dim), name='image_input')
  image_dense = Dense(n_units, activation='relu')(image_input)

  trans_embedding_input = Input(shape=(input_embedding_dim), name='text_embedding_input')
  trans_dense = Dense(n_units, activation = 'relu')(trans_embedding_input)

  word_input = Input(shape=(1, max_caption_length), name='word_input')
  lstm = LSTM(units=n_units, name='lstm')(word_input)

  # add all
  combined = add([image_dense, trans_dense, lstm])

  # decoder
  decoder = Dense(n_units, activation='relu')(combined)
  outputs = Dense(vocab_size, activation='softmax')(decoder)

  model = Model(inputs=[image_input, trans_embedding_input, word_input], outputs=outputs, name=model_name)

  return model



In [ ]:
model_name='baseline'
learning_rate = 0.001
epochs=10
batch_size=2

K.clear_session()

train_data, test_data, max_caption_length, vocab_size, input_image_dim, input_embedding_dim = get_dataset()

model = build_model(model_name=model_name, input_image_dim=input_image_dim, input_embedding_dim=input_embedding_dim, max_caption_length=max_caption_length, vocab_size=vocab_size)
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
loss = tf.keras.losses.categorical_crossentropy

model.summary()

model.compile(loss=loss, optimizer=optimizer, metrics=['accuracy'])

# Initialize a W&B run
wandb.init(
    project = 'instacap-demo',
    config = {
      "learning_rate": learning_rate,
      "epochs": epochs,
      "batch_size": batch_size,
      "model_name": model.name
    },
    name = model.name
)

start_time = time.time()
training_results = model.fit(
        train_data,
        validation_data=test_data,
        epochs=epochs,
        callbacks=[WandbCallback()],
        #callbacks = [WandbMetricsLogger(log_freq=1)],
        verbose=1)
execution_time = (time.time() - start_time)/60.0
print("Training execution time (mins)",execution_time)

# Update W&B
wandb.config.update({"execution_time": execution_time})
# Close the W&B run
wandb.run.finish()

env: GOOGLE_APPLICATION_CREDENTIALS=data_service_account.json
1/1 [==============================] - 2s 2s/step
Model: "baseline"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_input (InputLayer)    [(None, 2048)]               0         []                            
                                                                                                  
 text_embedding_input (Inpu  [(None, 768)]                0         []                            
 tLayer)                                                                                          
                                                                                                  
 word_input (InputLayer)     [(None, 1, 39)]              0         []                            
                                                                              

Epoch 1/10
4/4 [==============================] - ETA: 0s - loss: 6.5161 - accuracy: 0.0000e+00

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
wandb: Adding directory to artifact (/content/wandb/run-20231005_060222-t4qig5cn/files/model-best)... Done. 0.1s


4/4 [==============================] - 14s 3s/step - loss: 6.5161 - accuracy: 0.0000e+00 - val_loss: 5.5481 - val_accuracy: 0.0000e+00
Epoch 2/10
4/4 [==============================] - 0s 127ms/step - loss: 3.3094 - accuracy: 0.1429 - val_loss: 5.8501 - val_accuracy: 0.0000e+00
Epoch 3/10
4/4 [==============================] - 1s 265ms/step - loss: 1.8864 - accuracy: 0.2857 - val_loss: 6.7636 - val_accuracy: 0.0000e+00
Epoch 4/10
4/4 [==============================] - 0s 130ms/step - loss: 1.6857 - accuracy: 0.4286 - val_loss: 6.8528 - val_accuracy: 0.0179
Epoch 5/10
4/4 [==============================] - 0s 129ms/step - loss: 1.4971 - accuracy: 0.4286 - val_loss: 7.2125 - val_accuracy: 0.0089
Epoch 6/10
4/4 [==============================] - 0s 67ms/step - loss: 1.1077 - accuracy: 0.5714 - val_loss: 7.5157 - val_accuracy: 0.0000e+00
Epoch 7/10
4/4 [==============================] - 0s 57ms/step - loss: 0.9543 - accuracy: 0.8571 - val_loss: 7.5062 - val_accuracy: 0.0000e+00
Epoch 8/10


accuracy,▁▂▃▅▅▆█▇██
epoch,▁▂▃▃▄▅▆▆▇█
loss,█▄▃▃▂▂▂▁▁▁
val_accuracy,▁▁▁█▅▁▁▁▅▅
val_loss,▁▂▅▆▇██▆▅▄
accuracy,0.85714
best_epoch,0
best_val_loss,5.54814
epoch,9
loss,0.35806
val_accuracy,0.00893
